In [1]:
import xml.sax
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')
from collections import defaultdict as ddic
import numpy as np
from tqdm import tqdm
from Stemmer import Stemmer
from IPython.display import clear_output

[nltk_data] Downloading package stopwords to /home/anush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/anush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
titleID = {}
fileNo = 0
pageNo = 0
iMap = ddic(list)
offset = 0

stop_words = set(stopwords.words('english'))
stemmer = Stemmer('english')

In [3]:
filename = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
sergey = '../Sergey_brin.xml'

In [4]:
def writeFile(pageno, imap,fileno,titleoffset):
    
    global titleID
    prevoff = titleoffset
    stats = []
    for k in sorted(imap):
        initial = k + ' '
        fi =  imap[k]
        initial += ' '.join(fi)
        stats.append(initial)
    fname = '../index' + str(fileno) + '.txt'
    with open(fname,'w') as f:
        f.write('\n'.join(stats))
    stats = []
    statOffset = []
    
    for k in tqdm(sorted(titleID)):
        initial = str(k) + ' ' + titleID[k].strip()
        stats.append(initial)
        statOffset.append(str(prevoff))
        prevoff += len(initial) + 1
    
    with open('../title.txt','a') as f:
        f.write('\n'.join(stats))
        f.write('\n')
    
    with open('../titleOffset','a') as f:
        f.write('\n'.join(statOffset))
        f.write('\n')
        

In [5]:
def cleanupText(stats):
    global stop_words
    global stemmer
    
    stats = stats.encode("ascii", errors="ignore").decode()
    stats = re.sub(r'http[^\ ]*\ ',r' ',stats)
    stats = re.sub(r'&nbsp;|&lt;|&gt;|&amp;|&quot;|&apos;', r' ',stats)
    stats = re.sub(r'[^A-Za-z0-9]+',r' ',stats)
    stats = stats.split()
    stats = [i for i in stats if not i in stop_words]
    stats = stemmer.stemWords(stats)
    return stats

In [6]:
class Index():
    def __init__(self,refs,extlink,cat,body,info,title):
        self.title = title
        self.refs = refs
        self.extlink = extlink
        self.cat = cat
        self.body = body
        self.info = info
    def MakingDicts(self):
        total = ddic(int)
        
        title = ddic(int)
        for i in self.title:
            title[i] += 1
            total[i] += 1
        
        refs = ddic(int)
        for i in self.refs:
            refs[i] += 1
            total[i] += 1
            
        
        extlink = ddic(int)
        for i in self.extlink:
            extlink[i] += 1
            total[i] += 1
            
        cat = ddic(int) 
        for i in self.cat:
            cat[i] += 1
            total[i] += 1
            
        
        body = ddic(int)
        for i in self.body:
            body[i] += 1
            total[i] += 1
            
        info = ddic(int)
        for i in self.info:
            info[i] += 1
            total[i] += 1
        return (total,refs,extlink,cat,body,info,title)
    
    def IndexMapping(self,total,refs,extlink,cat,body,info,title,pageno,iMap):
        
        
        for word in total.keys():
            initial = 'D' + str(pageno)
            if refs[word] > 0:
                initial += 'r' + str(refs[word])
            if extlink[word] > 0:
                initial += 'e' + str(extlink[word])
            if cat[word] > 0:
                initial += 'c' + str(cat[word])
            if body[word] > 0:
                initial += 'b' + str(body[word])
            if info[word] > 0:
                initial += 'i' + str(info[word])
            if title[word] > 0:
                initial += 't' + str(title[word])
            
            iMap[word].append(initial)
        return iMap
    
    def InitIndex(self,pageno,iMap):
            
        (total,refs,extlink,cat,body,info,title) = self.MakingDicts()
        iMap = self.IndexMapping(total,refs,extlink,cat,body,info,title,pageno,iMap)
        
        return iMap, pageno
        

In [7]:
class RegexExp():
    def __init__(self):
        pass
    def findUsefulText(self, info):
        stats = info.split('\n')
        refs = []
        extlinks = []
        cat = []
        for l in stats:
            if re.search(r'ref',l):
                x = re.sub(r'.*title[\ ]*=[\ ]*([^\|]*).*',r'\1',l)
                refs.append(x)
            if re.search(r'\[\[category',l):
                x = re.sub(r'\[\[category:(.*)\]\]',r'\1',l)
                cat.append(x)
            if re.match(r'\*[\ ]*\[',l):
                extlinks.append(l)
        
        return (refs,extlinks,cat)
    
    def findUsefulInfo(self,info):
        stats = info.split('\n')
        i = []
        flag = 0
        
        for l in stats:
            if re.match(r'\{\{infobox', l):
                flag = 1
                x = re.sub('\{\{infobox(.*)',r'\1',l)
                i.append(x)
            elif flag == 1:
                if l == '}}':
                    flag = 0
                    continue
                i.append(l)
        empty = []
        stats = cleanupText(' '.join(i))
        return stats
                
        
    
    def textHandling(self,ID,text,title):
        text = text.lower()
        cleaned_text = text.split('==references==')
#         print(data)
        if len(cleaned_text) == 1:
            cleaned_text = text.split('== references== ')
        if len(cleaned_text) == 1:
            refs = []
            extlink = []
            cat = []
        else:
            useful = cleaned_text[1]
            (refs,extlink,cat) = self.findUsefulText(useful)
            refs = cleanupText(' '.join(refs))
            extlink = cleanupText(' '.join(extlink))
            cat = cleanupText(' '.join(cat))
        title = title.lower()
        useful = cleaned_text[0]
        info = self.findUsefulInfo(useful)
        stats = re.sub(r'\{\{.*\}\}',r' ',useful)
        body = cleanupText(stats)
        
        return (refs,extlink,cat,body,info,title)
        

In [8]:
class XMLHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData = ''
        self.title = ''
        self.ID = ''
        self.idFlag = 0
        self.text = ""
        self.pageNo = 0
        self.iMap = ddic(list)
        self.fileNo = 0
        
    def startElement(self,tag,attributes):
        self.CurrentData = tag
            
    def endElement(self,tag):
        if self.CurrentData == 'title':
            pass
#             print(self.title)
        elif self.CurrentData == 'text':
            pass
#             print(self.text)
        
        
        
        if tag == 'page':
            self.pageNo += 1
            clear_output(wait=True)
            print(self.pageNo)
            reg = RegexExp()
            refs,extlink,cat,body,info,title = reg.textHandling(self.ID,self.text,self.title)
            indexer = Index(refs,extlink,cat,body,info,title)
            self.iMap,self.pageNo = indexer.InitIndex(self.pageNo,self.iMap)
            
            global fileNo,pageNo,iMap,offset,titleID
            titleID[self.pageNo] = self.title.strip().encode("ascii",errors="ignore").decode()
            
            
            if self.pageNo%25000 == 0:
                offest = writeFile(self.pageNo, self.iMap,self.fileNo,offset)
                self.iMap = ddic(list)
                titleID = {}
                self.fileNo += 1
            
            fileNo = self.fileNo
            pageNo = self.pageNo
            iMap = self.iMap
            
            
            self.CurrentData = ''
            self.title = ''
            self.ID = ''
            self.idFlag = 0
            self.text = ''
            
        
        
    def characters(self,content):
        if self.CurrentData == 'title':
            self.title += content
        elif self.CurrentData == 'text':
            self.text += content
        elif self.CurrentData == 'id' and self.idFlag == 0:
            self.ID = content
            self.idFlag = 1

In [9]:
class Parser():
    def __init__(self,filename):
        self.parser = xml.sax.make_parser()
        self.parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        self.handler =  XMLHandler()
        self.parser.setContentHandler(self.handler)
        self.parser.parse(filename)

In [10]:
parser = Parser(filename)


19797


In [11]:
writeFile(pageNo, iMap,fileNo,offset)

100%|██████████| 19797/19797 [00:00<00:00, 573055.78it/s]
